In [2]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, RobertaTokenizer
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, RobertaModel

PRE_TRAINED_MODEL_NAME = 'microsoft/codebert-base'
tokenizer = RobertaTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = RobertaModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_json('data/variable_misuse/20200621_Python_variable_misuse_datasets_train.json', lines=True)
sample_code = df['function'][0:3]

encoding = tokenizer.batch_encode_plus(
  sample_code,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  padding=True,
  truncation=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [5]:
output = model(encoding['input_ids'], encoding['attention_mask'])

In [6]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [7]:
output['last_hidden_state'].shape

torch.Size([3, 32, 768])

In [8]:
cls_token = output['last_hidden_state'][:, 0, :]

In [9]:
cls_token.shape

torch.Size([3, 768])

In [10]:
cls_token

tensor([[-0.2165,  0.1640,  0.0849,  ..., -0.2742, -0.3673,  0.3376],
        [-0.2165,  0.1640,  0.0849,  ..., -0.2742, -0.3673,  0.3376],
        [-0.1627,  0.1377,  0.1339,  ..., -0.3064, -0.5019,  0.3590]],
       grad_fn=<SliceBackward0>)

In [11]:
from VariableMisuseClassifier import VariableMisuseClassifier

model = VariableMisuseClassifier(PRE_TRAINED_MODEL_NAME, 'roberta')

In [12]:
output = model(encoding['input_ids'], encoding['attention_mask'])

In [13]:
output

tensor([[0.4318],
        [0.4290],
        [0.4297]], grad_fn=<SigmoidBackward0>)